# Whiskas Cat Food

_Adapted from [material](https://twiki.esc.auckland.ac.nz/bin/view/OpsRes/WhiskasCatFoodProblem) by Dept. of Engineering Science, University of Auckland_

![](https://i.imgur.com/MlQqMz2.png)

Whiskas cat food is manufactured by Uncle Ben's. Uncle Ben's want to produce their cat food products as cheaply as possible while ensuring they meet the stated nutritional analysis requirements shown on the cans: 

![](https://i.imgur.com/dC2JgQV.jpg)

![](https://i.imgur.com/c3Xm7oI.jpg)

Thus they want to vary the quantities of each ingredient used (the main ingredients being chicken, beef, mutton, rice, wheat and gel) while still meeting their nutritional standards:

![](https://i.imgur.com/ox0gqH2.jpg)

The costs of the chicken, beef, and mutton are provided in the file `cost.csv`. (All costs are per 100g.) In this case study, we will ignore the vitamin and mineral ingredients. (Any costs for these are likely to be very small anyway.)

Each ingredient contributes to the total weight of protein, fat, fibre and salt in the final product. The contributions (in grams) per 100g of ingredient are given in the file `nutrition.csv`.

Our goal is to determine the optimal mix of ingredients to satisfy the nutritional standards at minimal cost.

## Problem formulation

### Variables

...


### Constraints

...

### Objective

...



## Reading in the data

The data for the problem has been given to us as two separate files, we'll need to read in and merge the data before we go further

In [23]:
using DataFrames
using DataFramesMeta

In [25]:
cost = readtable("cost.csv", header=false)

,x1,x2
1,Chicken,0.13
2,Beef,0.1
3,Mutton,0.1
4,Wheat,0.05
5,Rice,0.02
6,Gel,0.01


In [26]:
nutrition = readtable("nutrition.csv", header=false)

,x1,x2,x3,x4,x5
1,Chicken,10.0,6.0,0.1,0.2
2,Beef,20.0,12.0,0.4,0.5
3,Mutton,15.0,11.0,0.3,0.7
4,Rice,0.0,1.0,10.0,0.2
5,Wheat,4.0,1.0,15.0,0.8
6,Gel,0.0,0.0,0.0,0.0


Let's rename the columns to make it easier to work with

In [27]:
names!(cost, [:ingredient, :cost])
names!(nutrition, [:ingredient, :protein, :fat, :fibre, :salt])

,ingredient,protein,fat,fibre,salt
1,Chicken,10.0,6.0,0.1,0.2
2,Beef,20.0,12.0,0.4,0.5
3,Mutton,15.0,11.0,0.3,0.7
4,Rice,0.0,1.0,10.0,0.2
5,Wheat,4.0,1.0,15.0,0.8
6,Gel,0.0,0.0,0.0,0.0


Now we need to combine the tables into a single DataFrame. If the rows were in the same order, we could just merge the tables directly:

In [28]:
data = [cost nutrition[2:end]]

,ingredient,cost,protein,fat,fibre,salt
1,Chicken,0.13,10.0,6.0,0.1,0.2
2,Beef,0.1,20.0,12.0,0.4,0.5
3,Mutton,0.1,15.0,11.0,0.3,0.7
4,Wheat,0.05,0.0,1.0,10.0,0.2
5,Rice,0.02,4.0,1.0,15.0,0.8
6,Gel,0.01,0.0,0.0,0.0,0.0


But in our case, the rows are in different orders in each table. We should  combine the tables by _joining_ them with the ingredient as the ID.

In [29]:
data = join(cost, nutrition; on=:ingredient)

,ingredient,cost,protein,fat,fibre,salt
1,Beef,0.1,20.0,12.0,0.4,0.5
2,Chicken,0.13,10.0,6.0,0.1,0.2
3,Gel,0.01,0.0,0.0,0.0,0.0
4,Mutton,0.1,15.0,11.0,0.3,0.7
5,Rice,0.02,0.0,1.0,10.0,0.2
6,Wheat,0.05,4.0,1.0,15.0,0.8


We also need to encode the requirements

In [50]:
requirements = Dict(
    :protein => (8, Inf), 
    :fat     => (6, Inf),
    :fibre   => (0, 2),
    :salt    => (0, 0.4),
)

@show requirements[:fat][1]   # Lower limit on fat
@show requirements[:salt][2]  # Upper limit on salt

# Iterating over requirements
for (r, bounds) in requirements
    @show r
    @show bounds
end

(requirements[:fat])[1] = 6
(requirements[:salt])[2] = 0.4
r = :fibre
bounds = (0, 2)
r = :fat
bounds = (6, Inf)
r = :protein
bounds = (8, Inf)
r = :salt
bounds = (0, 0.4)


Great, now the data is together and correct!

We can now code up our optimization model:

In [48]:
using JuMP, Cbc
function find_optimal_mix(data, requirements)
    n = nrow(data)
    
    m = Model(solver=CbcSolver())
    
    @variable(m, x[1:n] >= 0)
    
    @objective(m, Min, sum(data[i, :cost] * x[i] for i = 1:n))
    
    for (r, bounds) in requirements
         @constraint(m, bounds[1] <= sum(data[i, r] * x[i] for i = 1:n) <= bounds[2])
    end

    @constraint(m, sum(x) == 1)
    
    solve(m)
    
    getvalue(x)
end

find_optimal_mix (generic function with 2 methods)

Now we can solve it!

In [47]:
mix = find_optimal_mix(data, requirements)

optimal_mix = DataFrame(ingredient=data[:ingredient], mix=mix)

requirements = Dict{Symbol,Any}(Pair{Symbol,Any}(:fibre, (0, 2)),Pair{Symbol,Any}(:fat, (6, Inf)),Pair{Symbol,Any}(:protein, (8, Inf)),Pair{Symbol,Any}(:salt, (0, 0.4)))


,ingredient,mix
1,Beef,0.49999999999999994
2,Chicken,0.0
3,Gel,0.5
4,Mutton,0.0
5,Rice,0.0
6,Wheat,0.0


## Exercise

What's the cost of this mix?

How can we present the results?

It's probably not reasonable that half of the food is just filler! Now let's suppose that Whiskas has a rule that only 30% of the contents can be gel. Modify the model and get the new optimal mix. How does it compare to the original solution?